In [ ]:
from flask import Flask, render_template, request, redirect, url_for,flash,session
from flask_mysqldb import MySQL
import os
import mysql.connector
import sys
import random
from flask import *
app = Flask(__name__)


app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''
app.config['MYSQL_DB'] = 'exam'

#dashboard
mysql = MySQL(app)
@app.route('/', methods=['GET', 'POST'])
def dashboard():
    return render_template('dashboard.html')

#teacher login
@app.route('/tlogin', methods=['GET', 'POST'])
def t_login():
    if request.method == "POST":
        details = request.form
        tid = details['tid']
        tpwd = details['tpwd']
        cur = mysql.connection.cursor()
        sql = "SELECT * from teacher where tid=%s and tpwd=%s"
        data = (tid,tpwd,)
        cur.execute(sql,data)
        myresult = cur.fetchall()
        if(cur.rowcount==1):
            session['tid']=tid
            return redirect(url_for('qbank'))
        else:
            flash('wrong id or password')
        cur.close()
    return render_template('t_login.html')

#create account for teacher
@app.route('/createacc',methods=['GET', 'POST'])
def createacc():
    if request.method == "POST":
        details = request.form
        tid = details['tid']
        tpwd = details['tpwd']
        tname = details['tname']
        tbranch = details['tbranch']
        cur = mysql.connection.cursor()
        sql = "SELECT * from teacher where tid=%s and tpwd=%s"
        data = (tid,tpwd,)
        cur.execute(sql,data)
        myresult = cur.fetchall()
        if(cur.rowcount==1):
            flash("User already there")
            return redirect(url_for('t_login'))
        else:
            cur.execute("INSERT INTO teacher (tid,tpwd,tname,branch) VALUES (%s, %s,%s,%s)", (tid,tpwd,tname,tbranch))
            mysql.connection.commit()
            return redirect(url_for('t_login'))
        cur.close()
    return render_template('t_signup.html')

#change password for teacher

@app.route('/change_t_password',methods=['GET', 'POST'])
def change_t_pwd():
    
    if request.method == "POST":
        details = request.form
        tid = session['tid']
        npwd = details['npwd']    #new password
        rnpwd = details['rnpwd']  #retyped new password
        cur = mysql.connection.cursor()
        sql = "SELECT * from teacher where tid=%s"
        data = (tid,)
        cur.execute(sql,data)
        result = cur.fetchall()
        cur.close()
        
        if (npwd==rnpwd):
            
            cur = mysql.connection.cursor()
            sql1 = "UPDATE teacher set tpwd=%s where tid=%s"
            data1 = (npwd,tid,)
            cur.execute(sql1,data1)
            mysql.connection.commit()
            result1 = cur.fetchall()
            cur.close()
            if(cur.rowcount==1):
                
                
                return redirect(url_for('logout'))
                flash("Successfully changed the password you have to relogin")
            else:
                flash("Error")
        else:
            
            flash("Passwords do not match please re enter")
        
        
    return render_template('student_change_password.html')

#student login page
@app.route('/slogin', methods=['GET', 'POST'])
def s_login():
    if request.method == "POST":
        details = request.form
        sid = details['sid']
        spwd = details['spwd']
        cur = mysql.connection.cursor()
        sql = "SELECT * from student where sid=%s and spwd=%s"
        data = (sid,spwd,)
        cur.execute(sql,data)
        myresult = cur.fetchall()
        if(cur.rowcount==1):
            session['sid']=sid
            return redirect(url_for('student_dashboard'))
        else:
            flash('wrong id or password or student does not exist')
        cur.close()
    return render_template('s_login.html')

#student signup
@app.route('/createaccstu',methods=['GET', 'POST'])
def createaccstu():
    if request.method == "POST":
        details = request.form
        sid = details['sid']
        spwd = details['spwd']
        sname = details['sname']
        sbranch = details['sbranch']
        sem = details['sem']
        cur = mysql.connection.cursor()
        sql = "SELECT * from student where sid=%s and spwd=%s"
        data = (sid,spwd,)
        cur.execute(sql,data)
        myresult = cur.fetchall()
        if(cur.rowcount==1):
            flash("User already there")
            return redirect(url_for('s_login'))
        else:
            cur.execute("INSERT INTO student (sid,spwd,sname,sbranch,sem) VALUES (%s, %s,%s,%s,%s)", (sid,spwd,sname,sbranch,sem))
            mysql.connection.commit()
            return redirect(url_for('s_login'))
        cur.close()
    return render_template('s_signup.html')

#student dashboard
@app.route('/studentdashboard',methods=['GET', 'POST'])
def student_dashboard():
    
    dasresult = []
    sid = session['sid']
    cur = mysql.connection.cursor()
    sql = "SELECT * from student where sid=%s"
    data = (sid,)
    cur.execute(sql,data)
    sresult = cur.fetchall()
    cur.close()
    sbranch = sresult[0][3]
    sem = sresult[0][4]
    cur = mysql.connection.cursor()
    sql1 = "SELECT cid from course where sem=%s and branch=%s"
    data2 = (sem,sbranch,)
    cur.execute(sql1,data2)
    result1 = cur.fetchall()
    #print(result1)
    cur.close()
    cur = mysql.connection.cursor()
    for h in result1:
        for l in h:
            
            sql2 = "SELECT * from quiz where cid=%s"
            data3 = (l,)
            cur.execute(sql2,data3)
            result2 = cur.fetchall()
            dasresult.append(result2)
            
    sql1 = "SELECT qid from result where sid=%s"
    data2 = (sid,)
    cur.execute(sql1,data2)
    quizes = cur.fetchall()
    #print(quizes)
    qlist=[]
    for i in quizes:
        qlist.append(i[0])
    dasresult_list=[]
    for i in dasresult:
        for j in i:
            if j[0] not in qlist:
                j = list(j)
                dasresult_list.append(j)
                
    #print(dasresult_list)
    test_taken=[]
    for i in qlist:
        sql2 = "SELECT * from quiz where qid=%s"
        data3 = (str(i),)
        cur.execute(sql2,data3)
        result2 = cur.fetchall()
        test_taken.append(result2)
    
    
    if request.method == "POST":
        if request.form.get("taketest"):
            details = request.form
            x = details['qid']
            return redirect(url_for('test_page',qid=x))
        
        if request.form.get("seeresult"):
            details = request.form
            x = details['qid']
            return redirect(url_for('result_page',qid=x))
       
        
    return render_template('student_dashboard.html',data1=sresult,data5=dasresult_list,test_taken=test_taken)

#change password for student
@app.route('/change_s_password',methods=['GET', 'POST'])
def change_s_pwd():
    
    if request.method == "POST":
        details = request.form
        sid = session['sid']
        npwd = details['npwd']    #new password
        rnpwd = details['rnpwd']  #retyped new password
        cur = mysql.connection.cursor()
        sql = "SELECT * from student where sid=%s"
        data = (sid,)
        cur.execute(sql,data)
        result = cur.fetchall()
        cur.close()
        
        if (npwd==rnpwd):
            
            cur = mysql.connection.cursor()
            sql1 = "UPDATE student set spwd=%s where sid=%s"
            data1 = (npwd,sid,)
            cur.execute(sql1,data1)
            mysql.connection.commit()
            result1 = cur.fetchall()
            cur.close()
            if(cur.rowcount==1):
                
                
                return redirect(url_for('logout'))
                flash("Successfully changed the password you have to relogin")
            else:
                flash("Error")
        else:
            
            flash("Passwords do not match please re enter")
        
        
    return render_template('student_change_password.html')

#general logout
@app.route('/logout',methods=['GET', 'POST'])
def logout():
    session.clear()
    return redirect(url_for('dashboard'))
    
que=[]
#page for question bank
@app.route('/qbank', methods=['GET', 'POST'])
def qbank():
    
    que.clear()
    tid=session['tid']
    i=" "
    cur = mysql.connection.cursor()
    sql="SELECT cname FROM `course` NATURAL JOIN `teaches` NATURAL JOIN `teacher` where tid="+str(tid)
    cur.execute(sql)
    data = cur.fetchall()
    cur.close()
    qdata1=""
    qdata2=""
    qdata3=""
    if session.get('cid') is not None:
        cur = mysql.connection.cursor()
        cid = session['cid']
        sql="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'"+" and type='1'"
        cur.execute(sql)
        qdata1 = cur.fetchall()
        #print(qdata1)
        sql="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'"+" and type='2'"
        cur.execute(sql)
        qdata2 = cur.fetchall()
        #print(qdata2)
        sql="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'"+" and type='3'"
        cur.execute(sql)
        qdata3 = cur.fetchall()
        #print(qdata3)
        cur.close()
        
        
    if request.method == "POST":
         
        if request.form.get("course"):
            cur = mysql.connection.cursor()
            details = request.form
            i = details['course']
            sql="SELECT cid FROM `course` WHERE cname='"+i+"'"
            cur.execute(sql)
            j=cur.fetchall()
            for val in j:
                for k in val:
                    cid=k
                    session['cid']=cid
                    sql1="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'"+" and type='1'"
                    sql2="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'"+" and type='2'"
                    sql3="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'"+" and type='3'"
            cur.execute(sql1)
            qdata1 = cur.fetchall()
            #print(qdata1)
            cur.execute(sql2)
            qdata2 = cur.fetchall()
            #print(qdata2)
            cur.execute(sql3)
            qdata3 = cur.fetchall()
            #print(qdata3)
            cur.close()
            
        elif request.form.get("add_que_type1"):
            details = request.form
            tid = session['tid']
            cid = session['cid']
            marks = details['marks']
            q = details['q']
            op1 = details['op1']
            op2 = details['op2']
            op3 = details['op3']
            op4 = details['op4']
            ans = details['ans']
            bt = details['bt']
            cur = mysql.connection.cursor()
            cur.execute("SELECT max(`qbid`) FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'")
            j=cur.fetchall()
            for val in j:
                for qbid in val:
                    if(qbid):
                        qbid=qbid+1
                    else:
                        qbid=1
            cur.execute("INSERT INTO `qbank`(`qbid`, `cid`, `tid`, `marks`, `que`, `op1`, `op2`, `op3`, `op4`, `ans`,`type`,`ans23`,`blooms_tax`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (qbid,cid,tid,marks,q,op1,op2,op3,op4,ans,'1','-1',bt))
            mysql.connection.commit()
            cur.close()
            return redirect(url_for('qbank'))
        
        elif request.form.get("add_que_type2"):
            details = request.form
            tid = session['tid']
            cid = session['cid']
            marks = details['marks']
            q = details['q']
            op1 = details['op1']
            op2 = details['op2']
            op3 = details['op3']
            op4 = details['op4']
            ans = details['ans']
            bt = details['bt']
            cur = mysql.connection.cursor()
            cur.execute("SELECT max(`qbid`) FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'")
            j=cur.fetchall()
            for val in j:
                for qbid in val:
                    if(qbid):
                        qbid=qbid+1
                    else:
                        qbid=1
            cur.execute("INSERT INTO `qbank`(`qbid`, `cid`, `tid`, `marks`, `que`, `op1`, `op2`, `op3`, `op4`, `ans23`,`type`,`ans`,`blooms_tax`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (qbid,cid,tid,marks,q,op1,op2,op3,op4,ans,'2','-1',bt))
            mysql.connection.commit()
            cur.close()
            return redirect(url_for('qbank'))
        
        elif request.form.get("add_que_type3"):
            details = request.form
            tid = session['tid']
            cid = session['cid']
            marks = details['marks']
            q = details['q']
            op1 = '-1'
            op2 = '-1'
            op3 = '-1'
            op4 = '-1'
            ans = details['ans']
            bt = details['bt']
            cur = mysql.connection.cursor()
            cur.execute("SELECT max(`qbid`) FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'")
            j=cur.fetchall()
            for val in j:
                for qbid in val:
                    if(qbid):
                        qbid=qbid+1
                    else:
                        qbid=1
            cur.execute("INSERT INTO `qbank`(`qbid`, `cid`, `tid`, `marks`, `que`, `op1`, `op2`, `op3`, `op4`, `ans23`,`type`,`ans`,`blooms_tax`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (qbid,cid,tid,marks,q,op1,op2,op3,op4,ans,'3','-1',bt))
            mysql.connection.commit()
            cur.close()
            return redirect(url_for('qbank'))
    return render_template('qbank.html',courses=data,ques1=qdata1,ques2=qdata2,ques3=qdata3,c=i)


@app.route('/viewreport',methods=['GET', 'POST'])
def view_report():
    
    tid=session['tid']
    i=" "
    cur = mysql.connection.cursor()
    sql="SELECT cname FROM `course` NATURAL JOIN `teaches` NATURAL JOIN `teacher` where tid="+str(tid)
    cur.execute(sql)
    data = cur.fetchall()
    cur.close()
    quizes=""
    report=""
    selected_test=""
    avg=""
    mn=""
    mx=""
    if request.method == "POST":
        if request.form.get("subject"):
            cur = mysql.connection.cursor()
            details = request.form
            i = details['course']
            sql="SELECT cid FROM `course` WHERE cname='"+i+"'"
            cur.execute(sql)
            j=cur.fetchall()
            for val in j:
                for k in val:
                    cid=k
                    session['cid']=cid
            sql="SELECT * FROM `quiz` WHERE tid="+str(tid)+" and cid='"+cid+"'"
            cur.execute(sql)
            quizes=cur.fetchall()
            #print(quizes)
            
        if request.form.get("view"):
            
            cur = mysql.connection.cursor()
            details = request.form
            qid = details['qid']
            cid = details['cid']
            sql="SELECT * FROM `result` WHERE qid='"+qid+"'"
            cur.execute(sql)
            report=cur.fetchall()
            #print(report)
            s=[]
            for j in report:
                s.append(int(j[2]))  
            if(len(s) != 0):
                avg = sum(s)/len(s)
                mx = max(s)
                mn = min(s)
            
            
                
            
            sql="SELECT * FROM `quiz` WHERE tid="+str(tid)+" and cid='"+cid+"'"
            cur.execute(sql)
            quizes=cur.fetchall()
            #print(quizes)
            
            for k in quizes:
                if(str(k[0]) == qid):
                    selected_test=k
                
            
            
    
    return render_template('view_report.html',courses=data,c=i,quizes=quizes,report=report,selected_test=selected_test,avg=avg,mn=mn,mx=mx)

que = []
@app.route('/cquizbutton', methods=['GET', 'POST'])
def c_quiz():
   
    tid=session['tid']
    i = " "
    cur = mysql.connection.cursor()
    sql="SELECT cname FROM `course` NATURAL JOIN `teaches` NATURAL JOIN `teacher` where tid="+str(tid)
    cur.execute(sql)
    data = cur.fetchall()
    cur.close()
    qdata1=""
    qdata2=""
    qdata3=""
    if session.get('cid') is not None:
        cur = mysql.connection.cursor()
        cid = session['cid']
        sql="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'"+" and type='1'"
        cur.execute(sql)
        qdata1 = cur.fetchall()
        #print(qdata1)
        sql="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'"+" and type='2'"
        cur.execute(sql)
        qdata2 = cur.fetchall()
        #print(qdata2)
        sql="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+cid+"'"+" and type='3'"
        cur.execute(sql)
        qdata3 = cur.fetchall()
        #print(qdata3)
        cur.close()
    if request.method == "POST":
         
        print(que)
        if request.form.get("course"):
            cur = mysql.connection.cursor()
            details = request.form
            i = details['course']
            sql="SELECT cid FROM `course` WHERE cname='"+i+"'"
            cur.execute(sql)
            j=cur.fetchall()
            for val in j:
                for k in val:
                    crs=k
                    session['cid']=crs
                    sql1="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"'"+" and type='1'"
                    sql2="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"'"+" and type='2'"
                    sql3="SELECT * FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"'"+" and type='3'"
            cur.execute(sql1)
            qdata1 = cur.fetchall()
            #print(qdata1)
            cur.execute(sql2)
            qdata2 = cur.fetchall()
            #print(qdata2)
            cur.execute(sql3)
            qdata3 = cur.fetchall()
            #print(qdata3)
            cur.close()
        elif request.form.get("add_que"):
            
            details = request.form
            
            m = details['qbid']
            if m not in que:
                que.append(m)
            
        elif request.form.get("remove_que"):
            
            details = request.form
            
            n = details['qbid']
            if n in que:
                que.remove(n)
                
        elif request.form.get("genq"):
            print("it comes here to go to genq")
            return redirect(url_for('gquiz',que=que))
            
    return render_template('c_quiz.html',courses=data,ques1=qdata1,ques2=qdata2,ques3=qdata3,c=i,que=que)


result3 = []
@app.route('/auto_test', methods=['GET', 'POST'])
def auto_test():
   
    tid=session['tid']
    cur = mysql.connection.cursor()
    sql="SELECT cname FROM `course` NATURAL JOIN `teaches` NATURAL JOIN `teacher` where tid="+str(tid)
    cur.execute(sql)
    data = cur.fetchall()
    cur.close()
    i=""
    blooms=""
    
    if request.method == "POST":
         
        
        if request.form.get("course"):
            cur = mysql.connection.cursor()
            details = request.form
            i = details['course']
            sql="SELECT cid FROM `course` WHERE cname='"+i+"'"
            cur.execute(sql)
            j=cur.fetchall()
            for val in j:
                for k in val:
                    crs=k
                    session['cid']=crs
                    sql1="SELECT `blooms_tax`,count(`blooms_tax`) FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"'"+" group by blooms_tax"
                
            cur.execute(sql1)
            blooms = cur.fetchall()
            print(blooms)
        
            
        if request.form.get("auto"):
            cur = mysql.connection.cursor()
            details = request.form
            i = details['co']
            ga=int(details['a'])
            gc=int(details['c'])
            ge=int(details['e'])
            gn=int(details['n'])
            gr=int(details['r'])
            gu=int(details['u'])
            #print(ga,gc,ge,gn,gr,gu)
            
            sql="SELECT cid FROM `course` WHERE cname='"+i+"'"
            cur.execute(sql)
            j=cur.fetchall()
            for val in j:
                for k in val:
                    crs=k
                    session['cid']=crs
                    #sql1="SELECT `blooms_tax`,count(`blooms_tax`) FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"'"+" group by blooms_tax"
                    sqla="SELECT `qbid` FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"' "+"and blooms_tax='A';"
                    sqlc="SELECT `qbid` FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"' "+"and blooms_tax='C';"
                    sqle="SELECT `qbid` FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"' "+"and blooms_tax='E';"
                    sqln="SELECT `qbid` FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"' "+"and blooms_tax='N';"
                    sqlr="SELECT `qbid` FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"' "+"and blooms_tax='R';"
                    sqlu="SELECT `qbid` FROM `qbank` WHERE tid="+str(tid)+" and cid='"+crs+"' "+"and blooms_tax='U';"
            
            cur.execute(sqla)
            a = cur.fetchall()
            al=[]
            qbids=""
            for k in a:
                al.append(k[0])
                
                
            cur.execute(sqlc)
            c = cur.fetchall()
            cl=[]
            for k in c:
                cl.append(k[0])
                
                
            cur.execute(sqle)
            e = cur.fetchall()
            el=[]
            for k in e:
                el.append(k[0])
                
            cur.execute(sqln)
            n = cur.fetchall()
            nl=[]
            for k in n:
                nl.append(k[0])
                
                
            cur.execute(sqlr)
            r = cur.fetchall()
            rl=[]
            for k in r:
                rl.append(k[0])
                
                
            cur.execute(sqlu)
            u = cur.fetchall() 
            ul=[]
            for k in u:
                ul.append(k[0])
            
            print(al,cl,el,nl,rl,ul)
            
            a_qbid = [] 
            a_num = random.sample(al,ga)
            for k in a_num:
                qbids=qbids+","+str(k)
                a_qbid.append(k)
            
            c_qbid = [] 
            c_num = random.sample(cl,gc)
            for k in c_num:
                qbids=qbids+","+str(k)
                c_qbid.append(k)
                
            e_qbid = [] 
            e_num = random.sample(el,ge)
            for k in e_num:
                qbids=qbids+","+str(k)
                e_qbid.append(k)
                
            n_qbid = [] 
            n_num = random.sample(nl,gn)
            for k in n_num:
                qbids=qbids+","+str(k)
                n_qbid.append(k)
                
            r_qbid = [] 
            r_num = random.sample(rl,gr)
            for k in r_num:
                qbids=qbids+","+str(k)
                r_qbid.append(k)
                     
            u_qbid = [] 
            u_num = random.sample(ul,gu)
            for k in u_num:
                qbids=qbids+","+str(k)
                u_qbid.append(k)
            
            que = qbids[1:]
            que = que.split(',')
            print("yep thats que",que)
            return redirect(url_for('gquiz',que=que))
            
    return render_template('auto_test.html',courses=data,blooms=blooms,c=i)

@app.route('/gquiz',methods=['GET', 'POST'])
def gquiz():
    #result3.clear()
    total_marks = 0
    tid = session['tid']
    cid = session['cid']
    cur = mysql.connection.cursor()
    if(request.args.getlist('que')):
        que= request.args.getlist('que')
        print("it comes here and here is que :", que)
        for j in que:
            sql = "SELECT marks from qbank where qbid=%s"
            cur.execute(sql,(j,))
            result = cur.fetchall()
            total_marks = total_marks + result[0][0]
        cur.close()
        cur = mysql.connection.cursor()
        for j in que:
            sql = "SELECT * from qbank where qbid=%s and tid=%s and cid=%s"
            cur.execute(sql,(j,tid,cid,))
            result1 = cur.fetchmany(size=2)
            result3.append(result1)
            #print(result3)
        cur.close()
   
    if request.method == "POST":
        details = request.form
        qid = details['quizid']
        qname = details['quizname']
        date = details['quizdate']
        stime = details['quizstime']
        etime = details['quizetime']
        ttime = details['quiztime']
        tmarks = details['quizmarks']
       
        
        question_string = ','.join(str(e) for e in que)
        qbids = question_string 
        cur = mysql.connection.cursor()
        cur.execute("INSERT INTO `quiz`(`qid`, `tid`, `cid`, `qname`, `qbids`, `tmarks`, `date`, `stime`, `etime`, `ttime`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(qid,tid,cid,qname,qbids,tmarks,date,stime,etime,ttime))
        mysql.connection.commit()
        
        sql1="SELECT sid FROM student as s,teaches as ts,teacher as tr,course as c WHERE s.sem = c.sem and s.sbranch=c.branch and ts.cid=c.cid and ts.tid=tr.tid and tr.tid=%s and c.cid=%s"
        data1=(tid,cid,)
        cur.execute(sql1,data1)
        sids = cur.fetchall()
        #print("here",sids)
        mysql.connection.commit()
        cur.close()
    return render_template('g_quiz.html',total = total_marks,ques=result3)

@app.route('/result_page', methods=['GET', 'POST'])
def result_page():
    
    qid = request.args.get('qid')
    sid = session['sid']
    cur = mysql.connection.cursor()
    sql = "SELECT * from quiz where qid=%s"
    data = (qid,)
    cur.execute(sql,data)
    fresult = cur.fetchall()
    
    
    sql1 = "SELECT `marks` from result where qid=%s and sid=%s"
    data1 = (qid,sid,)
    cur.execute(sql1,data1)
    check = cur.fetchall()
    m=-1
    
    if check:
        m=check[0][0]
    
    print(m)
        
    sql2 = " SELECT `cid` FROM `quiz` WHERE qid =%s"
    data2 = (qid,)
    cur.execute(sql2,data2)
    cid = cur.fetchall()
    cid=cid[0][0]
    #print(cid)
    
    sql2 = " SELECT * FROM `response` WHERE sid=%s and qid =%s"
    data2 = (sid,qid,)
    cur.execute(sql2,data2)
    response = cur.fetchall()
    #print(response)
    qbid=[]
    for i in response:
        qbid.append(i[2])
        
    ques12=[]
    ques3=[]
    #print(qbid)
    
    for i in qbid:
        temp=[]
        
        for j in response:
            if(j[2] == i):
                temp.append(list(j))
        temp=temp[0]
        
        
        sql2 = "  SELECT `que`, `type`, `op1`, `op2`, `op3`, `op4` FROM `qbank` WHERE qbid=%s and cid=%s"
        data2 = (i,cid,)
        cur.execute(sql2,data2)
        q = cur.fetchall()
        q= list(q[0])
        temp = temp+q
        if(int(q[1]) != 3):
            ques12.append(temp)
        else:
             ques3.append(temp)
    print(ques12)
    print(ques3)
       
        
    cur.close()

    return render_template("result_page.html",data=fresult,marks=m,ques12=ques12,ques3=ques3)


    

@app.route('/testpage', methods=['GET', 'POST'])
def test_page():
    
    qid = request.args.get('qid')
    sid = session['sid']
    cur = mysql.connection.cursor()
    sql = "SELECT * from quiz where qid=%s"
    data = (qid,)
    cur.execute(sql,data)
    fresult = cur.fetchall()
    
    
    sql1 = "SELECT `marks` from result where qid=%s and sid=%s"
    data1 = (qid,sid,)
    cur.execute(sql1,data1)
    check = cur.fetchall()
    m=-1
    cur.close()
    if check:
        m=check[0][0]
    
    
    
    if request.method == "POST":
    
        return redirect(url_for('test',qqid=qid))
    
    return render_template("test_page.html",data=fresult,marks=m)



@app.route('/test', methods=['GET', 'POST'])
def test():   
    qid = request.args.get('qqid')
    cur = mysql.connection.cursor()
    sql = "SELECT * from quiz where qid=%s"
    data = (qid,)
    cur.execute(sql,data)
    zresult = cur.fetchall()
    qbid = zresult[0][4]
    qbid = list(qbid)
    cid = zresult[0][2]
    tid = zresult[0][1]
    right_ans={}
    marks = {}
    right_ans23={}
    typ = {}
    for j in qbid:
        if(j==','):
            qbid.remove(',')
    for j in qbid:
        j=int(j)
        sql1="SELECT `ans`,`marks`,`ans23`,`type` FROM `qbank` WHERE qbid=%s and cid=%s and tid=%s;"
        data1 = (j,cid,tid,)
        cur.execute(sql1,data1)
        hresult = cur.fetchall()
        right_ans[j]=hresult[0][0]
        marks[j]=hresult[0][1]
        right_ans23[j]=hresult[0][2]
        typ[j]=hresult[0][3]
        
            
  
            
    #print(qbid)
    # DELETEing for our purpose
    ql = len(qbid)
    sid = session['sid']
    #sql = "DELETE from `response` where sid=%s and qid=%s and ans=%s"
    #data = (sid,qid,'0',)
    #cur.execute(sql,data)
    #mysql.connection.commit()
    index=[]
    index_val=1
    saved=[]
    for i in qbid:
        index.append(index_val)
        index_val=index_val+1
        sql = "SELECT * FROM `response`WHERE `sid`=%s and `qid`=%s and `qbid`=%s "
        data = (sid,qid,i,)
        cur.execute(sql,data)
        exist = cur.fetchall()
        if not exist:
            if(typ[int(i)] != 1):
                sql = "INSERT INTO `response`(`sid`, `qid`, `qbid`,`right_ans`,`marks`,`type`) VALUES (%s,%s,%s,%s,%s,%s)"
                data = (sid,qid,i,right_ans23[int(i)],marks[int(i)],typ[int(i)])
                cur.execute(sql,data)
                mysql.connection.commit()
            else:
                sql = "INSERT INTO `response`(`sid`, `qid`, `qbid`,`right_ans`,`marks`,`type`) VALUES (%s,%s,%s,%s,%s,%s)"
                data = (sid,qid,i,right_ans[int(i)],marks[int(i)],typ[int(i)])
                cur.execute(sql,data)
                mysql.connection.commit()
            
        sql = "SELECT `ans` FROM `response`WHERE `sid`=%s and `qid`=%s and `qbid`=%s "
        data = (sid,qid,i,)
        cur.execute(sql,data)
        var=cur.fetchall()
        saved.append(var[0][0])
    print(saved)    
    
    cid = zresult[0][2]
    tid = zresult[0][1]
    sql1 = "SELECT * from qbank where qbid=%s and cid=%s and tid=%s"
    data1 = (qbid[0],cid,tid,)
    cur.execute(sql1,data1)
    hresult = cur.fetchall()
    hresult = list(hresult[0])
    for i in index:
        if(int(qbid[i-1]) == int(hresult[0])):
            hresult.append(i)
            
    sql = "SELECT `ans` FROM `response`WHERE `sid`=%s and `qid`=%s and `qbid`=%s "
    data = (sid,qid,qbid[0],)
    cur.execute(sql,data)
    pre_sel = cur.fetchall()

    
    ct=0
    fs=0
    fc=0
    #print(index)
    
    if request.method == "POST":
        if(request.form.get("select")):
            details = request.form
            qid1 = details['select']
            ct=details['ct2']
            fs=details['fs2']
            fc=details['fc2']
            sql2 = "SELECT * from qbank where qbid=%s and cid=%s and tid=%s"
            data2 = (qbid[int(qid1)-1],cid,tid,)
            cur.execute(sql2,data2)
            hresult = cur.fetchall()
            hresult = list(hresult[0])
            for i in index:
                if(int(qbid[i-1]) == int(hresult[0])):
                    hresult.append(i)
            
            sql = "SELECT `ans` FROM `response`WHERE `sid`=%s and `qid`=%s and `qbid`=%s "
            data = (sid,qid,qbid[int(qid1)-1],)
            cur.execute(sql,data)
            pre_sel = cur.fetchall()
            saved=[]
            for i in qbid:
                sql = "SELECT `ans` FROM `response` WHERE `sid`=%s and `qid`=%s and `qbid`=%s "
                data = (sid,qid,i,)
                cur.execute(sql,data)
                var=cur.fetchall()
                saved.append(var[0][0])
            #print(saved)
            #print(qbid[int(qid1)-1])
            #print(pre_sel[0][0])
            #print("-----")
            
        if(request.form.get("save")):
            details = request.form
            ans = details['save']
            ct=details['ct3']
            fs=details['fs3']
            fc=details['fc3']
            #print(ans)
            ans = ans.split(' ')
            #print("ans0 : ",ans[0])
            #print("ans1 : ",ans[1])
            sql2 = "SELECT * from qbank where qbid=%s and cid=%s and tid=%s"
            data2 = (qbid[((int(ans[0]))% ql)],cid,tid,)
            cur.execute(sql2,data2)
            hresult = cur.fetchall()
            hresult = list(hresult[0])
            for i in index:
                if(int(qbid[i-1]) == int(hresult[0])):
                    hresult.append(i)
              
            sql = "SELECT `ans` FROM `response` WHERE `sid`=%s and `qid`=%s and `qbid`=%s "
            data = (sid,qid,qbid[((int(ans[0]))% ql)],)
            cur.execute(sql,data)
            pre_sel = cur.fetchall()
            #print(qbid[((int(ans[0]))% ql)])
            #print(pre_sel[0][0])
            #print("-----")
            
    
            sql3 = "UPDATE `response` SET `ans`=%s WHERE sid=%s and qid=%s and qbid=%s"
            data3 = (ans[1],sid,qid,qbid[(int(ans[0])-1)],)
            cur.execute(sql3,data3)
            mysql.connection.commit()
            saved=[]
            for i in qbid:
                sql = "SELECT `ans` FROM `response`WHERE `sid`=%s and `qid`=%s and `qbid`=%s "
                data = (sid,qid,i,)
                cur.execute(sql,data)
                var=cur.fetchall()
                saved.append(var[0][0])
            #print(saved)
        
        if(request.form.get("EndTest")):
            
            sql = "SELECT `qbid`,`ans`,`right_ans`,`marks`,`type` FROM `response` WHERE `sid`=%s and `qid`=%s "
            data = (sid,qid,)
            cur.execute(sql,data)
            response = cur.fetchall()
            marks=0
            for i in response:
                if(i[4] != 2):
                    if(int(i[1]) == int(i[2])):
                        sql3 = "UPDATE `response` SET `marks_given`=%s WHERE sid=%s and qid=%s and qbid=%s"
                        data3 = (int(i[3]),sid,qid,int(i[0]),)
                        cur.execute(sql3,data3)
                        mysql.connection.commit()
                        marks=marks+int(i[3])
                else:
                    print(i[1][1:],i[2])
                    a = i[1][1:]
                    a = a.split(",")
                    ar = i[2].split(",")
                    #print(a,ar)
                    a = set(a)  
                    ar = set(ar) 
                    if(a == ar):
                        sql3 = "UPDATE `response` SET `marks_given`=%s WHERE sid=%s and qid=%s and qbid=%s"
                        data3 = (int(i[3]),sid,qid,int(i[0]),)
                        cur.execute(sql3,data3)
                        mysql.connection.commit()
                        marks=marks+int(i[3])
                        
                    
                    
            print(marks)
            sql = "INSERT INTO `result`(`sid`, `qid`, `marks`) VALUES (%s,%s,%s)"
            data = (sid,qid,marks)
            cur.execute(sql,data)
            mysql.connection.commit()
            return redirect(url_for('result_page',qid=qid))
            
             
            
    cur.close()
    return render_template("test.html",data=zresult,row=hresult,ids=index,pre=pre_sel[0][0],time=zresult[0][6],ct=ct,fs=fs,fc=fc,saved=saved)



    
if __name__ == '__main__':
    app.secret_key = os.urandom(24)
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2022 06:40:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 06:40:35] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2022 06:40:40] "GET /tlogin HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 06:40:40] "GET /static/img_avatar2.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 06:40:43] "POST /tlogin HTTP/1.1" 302 -
127.0.0.1 - - [14/Mar/2022 06:40:43] "GET /qbank HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 06:40:50] "POST /qbank HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 06:40:54] "GET /viewreport HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 06:40:57] "POST /viewreport HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 06:41:00] "POST /viewreport HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 06:41:31] "GET /qbank HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 06:41:45] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [14/Mar/2022 06:41:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2022 06:41:46] "GET /slogin HTTP/1.1" 

5
[[1, 22, 2, '2', 1, '2', 1, 1, 'Which is not the use of divider?', 1, 'To divide curved or straight lines into the desired number of equal parts', 'To draw circles', 'To transfer dimensions from one part of the drawing to another part', 'To set-off given distances from the scale to the drawing'], [1, 22, 3, '3', 1, '3', 1, 1, ' The areas of the two subsequent sizes of drawing sheet are in the ratio ____', 1, '1:5', '1:4', '1:2', '1:10'], [1, 22, 5, ',1,2', 2, '1,2', 2, 2, 'solution for x^2 - 1 = 0 ?', 2, '1', '-1', '0', 'none'], [1, 22, 6, '2', 0, '1', 1, 1, 'what is eg?', 1, 'a', 'b', 'c', 'd'], [1, 22, 9, '3', 0, '1', 1, 1, 'This type of projection is when projectors are parallel to each other, but are at an angle other than 90 degrees to the plane of projection:', 1, 'Oblique projection', 'Perpendicular projection', 'Aesthetic projection', 'Angular projection']]
[[1, 22, 4, '9', 1, '9', 1, 3, 'what is 3^2 ?', 3, '-1', '-1', '-1', '-1']]


127.0.0.1 - - [14/Mar/2022 06:42:02] "GET /studentdashboard HTTP/1.1" 200 -
